## Imports

In [169]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Point

## Read-in

In [138]:
df = pd.read_csv('CookCountyFeb-Mar Foreclosures - feb_mar_foreclosures.csv')

In [139]:
df = df.drop(columns=['Color','COLOR'])

## Clean, drop, and convert

In [140]:
df['Doc Recorded'] = pd.to_datetime(df['Doc Recorded'])

## Data Stuff

In [141]:
# Count number of February dates
feb_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 2])

print(f'Number of February dates: {feb_dates_count}')

Number of February dates: 662


In [142]:
# Count number of February dates
mar_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 3])

print(f'Number of March dates: {mar_dates_count}')

Number of March dates: 778


In [143]:
df = df.drop(columns='1st PIN')

In [144]:
df['1st Grantor'] = df['1st Grantor'].fillna('NA')
df['1st Grantee'] = df['1st Grantee'].fillna('NA')

In [145]:
municipal_authority = ['city', 'town', 'municipality', 'village','transit auth','department of transp']  # list of municipal authority keywords
df = df[~df['1st Grantor'].str.contains('|'.join(municipal_authority), case=False) & 
        ~df['1st Grantee'].str.contains('|'.join(municipal_authority), case=False)]

In [146]:
# create new column and assign colors based on month
df['COLOR'] = df['Doc Recorded'].apply(lambda x: 'orange' if x.month == 2 else 'red' if x.month == 3 else '')

## Get mortgage amounts

In [147]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'
}

In [148]:
def mortgage_url_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    for link in soup.find_all('a', href=True):
        if link['href'].startswith('/Document/Detail'):
            mortgage_url = 'https://crs.cookcountyclerkil.gov' + link['href']
            return mortgage_url

In [149]:
def mortgage_consi_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find all tr tags
    trs = soup.find_all('tr')

    # Loop through each tr tag and look for the td tag containing 'Consideration Amount' label
    for tr in trs:
        td = tr.find('th', text='Consideration Amount:')
        if td:
            # If the td tag is found, get the next td tag containing the amount
            amount_td = td.find_next_sibling('td')
            if amount_td:
                # Print the amount
                amount = amount_td.text.strip()
                return amount
            else:
                return 'not found'

In [150]:
df['mortgage_urls'] = df['deed_urls'].apply(lambda x: mortgage_url_snagger(x, headers))

In [151]:
df['mortgage_amount'] = df['mortgage_urls'].apply(lambda x: mortgage_consi_snagger(x, headers) if x is not None else None)

In [152]:
df[['mortgage_urls','mortgage_amount']] = df[['mortgage_urls','mortgage_amount']].fillna('NA')

## HTML Popup Formatter

In [153]:
df.columns

Index(['Doc Number', 'Doc Recorded', 'Doc Executed', 'Doc Type', 'Consi. Amt.',
       '1st Grantor', '1st Grantee', 'Assoc. Doc#', 'deed_urls',
       'folio_number', 'address', 'geo_address', 'geocoded', 'lat', 'lon',
       'COLOR', 'mortgage_urls', 'mortgage_amount'],
      dtype='object')

In [154]:
def popup_html(row):
    grantor = row['1st Grantor']
    grantee = row['1st Grantee']
    folio = row['folio_number']
    address = row['address']
    mortgage_amount = row['mortgage_amount']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Lendor: </strong>{}'''.format(grantor) + '''<br>
    <strong>Lendee: </strong>{}'''.format(grantee) + '''<br>
    <strong>PIN: </strong>{}'''.format(folio) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Mortgage Amount: </strong>{}'''.format(mortgage_amount) + '''<br>
    </html>
    '''
    return html

In [254]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Cook County Pending Foreclosures')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February & March')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_orange = folium.FeatureGroup(name='February')
fg_red = folium.FeatureGroup(name='March')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'orange':
        marker = folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            color='orange',
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)
    else:
        marker = folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            color=row['COLOR'],
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_red)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_red.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
folium.TileLayer('CartoDBpositron',control=False).add_to(m) 
            
# Display map
m

In [255]:
m.save('index.html')

## Map URL Snagger

In [157]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/lis_pendens_scraper


## Stats for story

In [158]:
# Count number of February dates
feb_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 2])

print(f'Number of February dates: {feb_dates_count}')

# Count number of February dates
mar_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 3])

print(f'Number of March dates: {mar_dates_count}')

Number of February dates: 662
Number of March dates: 770


In [159]:
feb = df.loc[df['Doc Recorded'].dt.month == 2]
mar = df.loc[df['Doc Recorded'].dt.month == 3]

In [194]:
39 + 37 + 28

104

In [191]:
feb['1st Grantor'].value_counts().head(60)

NATIONSTAR MTG LLC                    40
US BK NATL ASSN                       39
US BK NATL ASSN TR                    37
LAKEVIEW LOAN SERVICING LLC           35
NEWREZ LLC                            32
US BK TRUST NATL ASSN TR              28
WILMINGTON SAV FUND SOC FSB TR        26
WELLS FARGO BK NA                     21
FREEDOM MTG CORP                      18
PENNYMAC LOAN SERVICES LLC            17
CARRINGTON MTG SERVICES LLC           16
MIDFIRST BK                           14
DEUTSCHE BK NATL TRUST CO TR          13
LOANDEPOTCOM LLC                      11
JPMORGAN CHASE BK NATL ASSN           10
PNC BK NATL ASSN                       9
THE BK OF NEW YORK MELLON TR           9
ROCKET MTG LLC                         7
FEDERAL HOME LOAN MTG CORP TR          7
FIFTH THIRD BK NATL ASSN               7
UNITED WHOLESALE MTG LLC               6
CITIZENS BK NA                         5
GOLBERT CHARLES P GDN                  5
MORTGAGE ASSETS MGMT LLC               5
SELECT PORTFOLIO

In [193]:
56 + 42 + 36

134

In [192]:
mar['1st Grantor'].value_counts().head(60)

US BK NATL ASSN                                56
LAKEVIEW LOAN SERVICING LLC                    46
US BK NATL ASSN TR                             42
NATIONSTAR MTG LLC                             39
US BK TRUST NATL ASSN TR                       36
NEWREZ LLC                                     32
FREEDOM MTG CORP                               30
WILMINGTON SAV FUND SOC FSB TR                 27
WELLS FARGO BK NA                              21
FIFTH THIRD BK NATL ASSN                       20
PENNYMAC LOAN SERVICES LLC                     19
CARRINGTON MTG SERVICES LLC                    18
FEDERAL HOME LOAN MTG CORP TR                  16
DEUTSCHE BK NATL TRUST CO TR                   13
LOANDEPOTCOM LLC                               13
THE BK OF NEW YORK MELLON TR                   12
JPMORGAN CHASE BK NATL ASSN                    10
ROCKET MTG LLC                                 10
BANK OF AMER NA                                 9
PNC BK NATL ASSN                                7


In [163]:
mar['1st Grantee'].value_counts()

CHICAGO TITLE LAND TRUST CO TR    5
UNKNOWN                           4
SMITH JAMES                       3
DELEE HOLDINGS LLC                3
JEBB FOOD SERVICES INC            2
                                 ..
CHANDLER DIONISIO                 1
WINES DOROTHY                     1
SALGADO PEDRO                     1
MCBRIDE MONIQUE D                 1
MARTIN ELIZABETH                  1
Name: 1st Grantee, Length: 753, dtype: int64

In [164]:
df.to_csv('feb_mar_foreclosures.csv')

In [249]:
mar['mortgage_amount_int'] = mar['mortgage_amount'].str.replace('$','')
mar['mortgage_amount_int'] = mar['mortgage_amount_int'].str.replace(',','')
mar = mar.loc[mar['mortgage_amount_int'] != 'NA']
mar['mortgage_amount_int'] = mar['mortgage_amount_int'].astype(float).fillna(0).astype(int)
mar['mortgage_amount_int'].median()

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_34497/1868218194.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  mar['mortgage_amount_int'] = mar['mortgage_amount'].str.replace('$','')


166822.0

In [251]:
feb['mortgage_amount_int'] = feb['mortgage_amount'].str.replace('$','')
feb['mortgage_amount_int'] = feb['mortgage_amount_int'].str.replace(',','')
feb = feb.loc[feb['mortgage_amount_int'] != 'NA']
feb['mortgage_amount_int'] = feb['mortgage_amount_int'].astype(float).fillna(0).astype(int)
feb['mortgage_amount_int'].median()

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_34497/837379293.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  feb['mortgage_amount_int'] = feb['mortgage_amount'].str.replace('$','')
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_34497/837379293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb['mortgage_amount_int'] = feb['mortgage_amount'].str.replace('$','')
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_34497/837379293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

162243.0

## Boundary Analysis

In [179]:
boundaries = gpd.read_file('Boundaries - Neighborhoods.geojson')

In [201]:
# create Point objects from lat/lon columns in df
geometry = gpd.points_from_xy(mar['lon'], mar['lat'])

# create geodataframe from df with Point objects as geometry
gdf_filings_mar = gpd.GeoDataFrame(mar, geometry=geometry)

gdf_filings_mar.crs = 'EPSG:4326'
gdf_filings_mar = gdf_filings_mar.to_crs(boundaries.crs)

# use contains method to get count of points within each neighborhood
counts_mar = gpd.sjoin(gdf_filings_mar, boundaries, predicate='within').groupby('pri_neigh').size().reset_index(name='count')

# create Point objects from lat/lon columns in df
geometry = gpd.points_from_xy(feb['lon'], feb['lat'])

# create geodataframe from df with Point objects as geometry
gdf_filings_feb = gpd.GeoDataFrame(feb, geometry=geometry)

gdf_filings_feb.crs = 'EPSG:4326'
gdf_filings_feb = gdf_filings_feb.to_crs(boundaries.crs)

# use contains method to get count of points within each neighborhood
counts_feb = gpd.sjoin(gdf_filings_feb, boundaries, predicate='within').groupby('pri_neigh').size().reset_index(name='count')